In [1]:
import sys 
import os  
import json
import datetime as dt 
import pandas as pd 
import numpy as np 
import talib
import sqlite3 as sql
from itertools import repeat
from p_tqdm import p_umap
sys.path.append('../')
from backtester import * 
from fxcmtoken import my_assets

In [2]:
for i, a in enumerate(my_assets):
    print(i, a)

0 EUR/USD
1 USD/JPY
2 GBP/USD
3 USD/CHF
4 AUD/USD
5 USD/CAD
6 NZD/USD
7 Bund
8 AUS200
9 ESP35
10 EUSTX50
11 FRA40
12 GER30
13 HKG33
14 JPN225
15 NAS100
16 SPX500
17 UK100
18 US30
19 Copper
20 NGAS
21 UKOil
22 USOil
23 XAU/USD
24 XAG/USD


In [11]:
freq = 'H1'
conn = sql.connect(f"/home/kachain/python_projects/algorithmic_trading/PriceData/PriceData_{freq}.db")
ticker = my_assets[1]
print(ticker)
data_sql = pd.read_sql(f"SELECT * FROM '{ticker}'", conn, parse_dates=['date'], index_col=['date'])
for col in ['open', 'close', 'high', 'low']:
    data_sql[col] = data_sql[['bid'+col, 'ask'+col]].mean(axis=1)
conn.close()
data_sql.tail()

USD/JPY


,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,tickqty,open,close,high,low
date,,,,,,,,,,,,,
2022-08-19 16:00:00,137.016,136.868,137.016,136.830,137.030,136.881,137.030,136.846,8612,137.0230,136.8745,137.0230,136.8380
2022-08-19 17:00:00,136.867,136.795,136.896,136.750,136.880,136.812,136.909,136.763,6561,136.8735,136.8035,136.9025,136.7565
2022-08-19 18:00:00,136.796,136.883,136.893,136.731,136.812,136.896,136.905,136.750,6376,136.8040,136.8895,136.8990,136.7405
2022-08-19 19:00:00,136.882,136.784,136.882,136.775,136.895,136.801,136.896,136.794,4209,136.8885,136.7925,136.8890,136.7845
2022-08-19 20:00:00,136.784,136.857,136.906,136.765,136.803,136.983,136.983,136.781,2879,136.7935,136.9200,136.9445,136.7730


In [30]:
def get_mama_return_lastweek(n_period, data, freq):
    
    data['ht'] = talib.HT_TRENDLINE(data.close)
    data.dropna(axis=0, inplace=True)
    data['mama'], data['fama'] = talib.MAMA(data.ht, fastlimit=n_period, slowlimit=n_period/10)
    data.dropna(axis=0, inplace=True)
    data['signals'] = np.where(data.mama > data.fama, 1, -1) 
    
    sat_index = data.resample('W-Sat').last().index
    data = data.loc[sat_index[-2]:sat_index[-1], :]
    
    backtester = IterativeBacktester(data=data, signals=data.signals, freq=freq)
    backtester.backtest(progress_bar=False)

    return n_period, backtester.return_df.loc['TotalReturn', 'Portfolio']


def get_tsf_return_lastweek(n_period, data, freq):
    
    data['ht'] = talib.HT_TRENDLINE(data.close)
    data['tsf'] = talib.TSF(data.ht, n_period)
    data.dropna(axis=0, inplace=True)
    data['signals'] = np.where(data.ht > data.tsf, 1, -1)
    
    sat_index = data.resample('W-Sat').last().index
    data = data.loc[sat_index[-2]:sat_index[-1], :]
    
    backtester = IterativeBacktester(data=data, signals=data.signals, freq=freq)
    backtester.backtest(progress_bar=False)

    return n_period, backtester.return_df.loc['TotalReturn', 'Portfolio']

def get_slope_return_lastweek(n_period, data, freq):
    
    data['ht'] = talib.HT_TRENDLINE(data.close)
    data.dropna(axis=0, inplace=True)
    data['signals'] = talib.LINEARREG_SLOPE(data.ht, timeperiod=n_period).apply(np.sign)
    data.dropna(axis=0, inplace=True)
    
    sat_index = data.resample('W-Sat').last().index
    data = data.loc[sat_index[-2]:sat_index[-1], :]
    
    backtester = IterativeBacktester(data=data, signals=data.signals, freq=freq)
    backtester.backtest(progress_bar=False)

    return n_period, backtester.return_df.loc['TotalReturn', 'Portfolio']

In [34]:
mama_res = p_umap(get_mama_return_lastweek, np.arange(0.1, 1, 0.1),
                  repeat(data_sql.copy(deep=True)), 
                  repeat(freq),
                  leave=False)
best_mama = pd.DataFrame(mama_res).sort_values(by=1, ascending=False).iloc[0,0]
best_mama

0.8

In [40]:
slope_res = p_umap(get_slope_return_lastweek, 
                       np.arange(10, 100, 10),
                       repeat(data_sql.copy(deep=True)), 
                       repeat(freq),
                       leave=False)
best_slope = pd.DataFrame(slope_res).sort_values(by=1, ascending=False).iloc[0, 0]
best_slope

70

In [38]:
tsf_res = p_umap(get_tsf_return_lastweek, 
                 np.arange(10, 100, 10),
                 repeat(data_sql.copy(deep=True)), 
                 repeat(freq),    
                 leave=False)
best_tsf = pd.DataFrame(tsf_res).sort_values(by=1, ascending=False).iloc[0,0]
best_tsf

90

90